<a href="https://colab.research.google.com/github/tazar09/heroes_3/blob/main/h3_heroes%5B27.oct2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [67]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

# Importing data

In [68]:
df1 = pd.read_html('https://heroes.thelazy.net/index.php/List_of_heroes_(HotA)')
df1[0].head()

,Name,Class,Unnamed: 2,Specialty,Unnamed: 4,Skill 1,Unnamed: 6,Skill 2,Unnamed: 8,Spell
0,Anabel,Captain,NaN,Pirates,NaN,Basic Offense,NaN,Basic Archery,NaN,NaN
1,Cassiopeia,Captain,NaN,Nymphs,NaN,Basic Offense,NaN,Basic Tactics,NaN,NaN
2,Corkes,Captain,NaN,Offense,NaN,Basic Offense,NaN,Basic Pathfinding,NaN,NaN
3,Derek,Captain,NaN,Crew Mates,NaN,Basic Offense,NaN,Basic Leadership,NaN,NaN
4,Elmore,Captain,NaN,Navigation,NaN,Advanced Navigation,NaN,NaN,NaN,NaN


# Creating dataframe

## Data preprocessing

In [69]:
df = df1[0].copy()
df = df.drop(axis = 1, labels = ['Unnamed: 2','Unnamed: 4', 'Unnamed: 6', 'Unnamed: 8'])
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell']

## Dropping and renaming

In [70]:
skill1 = df['skill_1'].str.split(expand = True, n = 1)
skill2 = df['skill_2'].str.split(expand = True, n = 1)
df = pd.concat([df, skill1, skill2], axis = 1)
df.columns = ['name', 'class', 'specialty', 'skill_1', 'skill_2', 'spell', 'skill_1_level', 'skill_1_name', 'skill_2_level', 'skill_2_name']

In [71]:
df = df.drop(axis = 1, labels = ['skill_1', 'skill_2'])

In [72]:
df = df[['name', 'class', 'specialty', 'skill_1_name', 'skill_1_level', 'skill_2_name','skill_2_level', 'spell']]
df['spell'] = df['spell'].fillna('')
df['skill_2_name'] = df['skill_2_name'].fillna('')
df['skill_2_level'] = df['skill_2_level'].fillna('')

## Creating new columns.

In [73]:
df['count'] = 1
df['spell_tf'] = df['spell'].apply(lambda x: 0 if x == '' else 1)

## Data from other sources

### Data from Towns doc

In [74]:
towns = pd.read_html('http://heroes.thelazy.net/index.php/List_of_all_town_names')
towns = list(towns[1]['Town'].unique())

In [75]:
towns = ['Castle', 'Rampart', 'Tower', 'Inferno', 'Necropolis','Dungeon', 'Stronghold' ,'Fortress','Conflux', 'Cove']

In [76]:
classes = list(df['class'].unique())
classes.remove('Captain')
classes.remove('Navigator')
classes.insert(18, 'Captain')
classes.insert(19, 'Navigator')
classes[8] = 'Death Knight'

In [77]:
towns2 = []
temp_list = []
for i in towns:
  temp_list = [i, i]
  towns2.append(temp_list)
towns2x = [element for sublist in towns2 for element in sublist]
len(classes), len(towns2x)

(20, 20)

In [78]:
class_towns = pd.DataFrame({'class': classes, 'town': towns2x})
df = df.merge(class_towns, how='left', on='class')

In [79]:
df.columns = ['name', 'class', 'specialty', 'skill_1', 'level_1',
       'skill_2', 'level_2', 'spell', 'count', 'spell_tf', 'town']

In [80]:
df['class_2'] = df['skill_1'].apply(lambda x: 'magic' if x == 'Wisdom' else 'might')
df.iloc[81, 11] = 'might'  #Rashka
df.iloc[101, 11] = 'magic' #Aislinn

In [81]:
df = df[['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1',
       'level_2', 'skill_2', 'spell', 'count', 'spell_tf', 'town']]

In [82]:
df.iloc[91:101 , 11] = 'Necropolis'

### Data from creatures_csv

In [83]:
creatures = pd.read_csv('/content/df_h3.csv')
creatures = creatures[['Name', 'Town','Upgraded']]

In [84]:
creatures['specialty'] = creatures['Name'].apply(lambda x: x + 's')
creatures = creatures[creatures['Upgraded'] == 0].reset_index(drop = True)

In [85]:
creatures['specialty'] = creatures['specialty'].apply(lambda x: 'convert' if x.endswith('mans') else x)

## Manually correcting errors

In [86]:
# creatures[creatures['specialty'] == 'convert']
creatures.iloc[0, 3] = 'Pikemen'
creatures.iloc[3, 3] = 'Swordsmen'
creatures.iloc[50, 3] = 'Lizardmen'

creatures.iloc[64, 3] = 'Crew Mate'
creatures.iloc[65, 3] = 'Sea Dogs'
creatures.iloc[67, 3] = 'Sea Witches'
creatures.iloc[68, 3] = 'Nix'

creatures.iloc[8, 3] = 'Dwarves'
creatures.iloc[9, 3] = 'Elves'
creatures.iloc[10, 3] = 'Pegasi'
creatures.iloc[26, 3] = 'Efreet'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[32, 3] = 'Liches'
creatures.iloc[36, 3] = 'Harpies'
creatures.iloc[47, 3] = 'Cyclops'
creatures.iloc[51, 3] = 'Cyclops'

creatures.iloc[11, 3] = 'Dendroids'
creatures.iloc[16, 3] = 'Golems'
creatures.iloc[17, 3] = 'Magi'

# creatures.head(5)

## Merging data (df1)

## *Column on creature specialty*

In [87]:
df2 = df.merge(creatures, how = 'left', on = 'specialty', suffixes = None)
df2 = df2.drop(axis = 0, index = 135).reset_index(drop = True)

## Manually correcting errors

In [88]:
df2.iloc[73,12] = 'Enchanter'
df2.iloc[0,12] = 'Pirate'
df2.iloc[3,12] = 'Crew Mate'
df2.iloc[9,12] = 'Sea Dog'
df2.iloc[96,12] = 'Walking Dead'
df2.iloc[150, 12] = 'Serpent Fly'
df2 = df2.drop(axis = 0, index = 135).reset_index(drop = True)

In [89]:
df2['Name'] = df2['Name'].fillna('')
df2 = df2.drop(axis=1, labels = ['Town', 'Upgraded', 'count', 'spell_tf'])

In [90]:
df2.columns = ['name', 'class', 'class_2', 'specialty', 'level_1', 'skill_1', 'level_2', 'skill_2', 'spellbook', 'town', 'skill_creature']

In [173]:
df2.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spellbook,town,skill_creature
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,


## Recreating (df)

In [174]:
df = df2.copy()
df.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spellbook,town,skill_creature
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,


## *Column on resource specialty*

In [192]:
df['skill_resource'] = ''
df.head()

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spellbook,town,skill_creature,skill_resource
0,Anabel,Captain,might,Pirates,Basic,Offense,Basic,Archery,,Cove,Pirate,
1,Cassiopeia,Captain,might,Nymphs,Basic,Offense,Basic,Tactics,,Cove,Nymph,
2,Corkes,Captain,might,Offense,Basic,Offense,Basic,Pathfinding,,Cove,,
3,Derek,Captain,might,Crew Mates,Basic,Offense,Basic,Leadership,,Cove,Crew Mate,
4,Elmore,Captain,might,Navigation,Advanced,Navigation,,,,Cove,,


In [193]:
pattern_resource = '|'.join(['Crystal', 'Mercury', 'Gold', 'Sulfur', 'Gems'])
index_resource = df[df['specialty'].str.contains(pattern_resource)].index.values

In [ ]:
df.iloc[index_resource, [11]] = df.iloc[index_resource, [3]].values.flatten().reshape(15,1)

In [207]:
print(df['specialty'].unique())

['Pirates' 'Nymphs' 'Offense' 'Crew Mates' 'Navigation' 'Stormbirds'
 'Cannon' 'Gold' 'Scouting' 'Sea Dogs' 'Nix' 'Crystal' 'Cure'
 'Sea Witches' 'Air Shield' 'Clone' 'Fireball' 'Sorcery' 'Forgetfulness'
 'Ballista' 'Griffins' 'Archery' 'Swordsmen' 'Cavaliers' 'Archers'
 'Estates' 'Speed' 'Bless' 'Frost Ring' 'Weakness' 'Monks' 'Prayer'
 'First Aid' 'Eagle Eye' 'Unicorns' 'Elves' 'Logistics' 'Armorer'
 'Dendroids' 'Resistance' 'Dwarves' 'Sharpshooters' 'Interference'
 'Pegasi' 'Ice Bolt' 'Slayer' 'Intelligence' 'Fortune' 'Nagas' 'Genies'
 'Golems' 'Gargoyles' 'Mercury' 'Hypnotize' 'Haste' 'Mysticism'
 'Chain Lightning' 'Magi' 'Enchanters' 'Gogs' 'Hell Hounds' 'Imps'
 'Demons' 'Pit Fiends' 'Efreet' 'Devils' 'Bloodlust' 'Sulfur'
 'Inferno (spell)' 'Wights' 'Skeletons' 'Necromancy' 'Liches'
 'Walking Dead' 'Black Knights' 'Vampires' 'Meteor Shower' 'Death Ripple'
 'Animate Dead' 'Stone Skin' 'Beholders' 'Minotaurs' 'Harpies'
 'Troglodytes' 'Manticores' 'Dragons' 'Resurrection' 'Goblins' '

## *Column on secondary skill specialty*

Some heroes specialize in a secondary skill, like Necromancy, Armorer, Navigation, Offense etc.

## *Column on spell skill specialty*

Some heroes specialize in casting a skill, like Ressurection, Inferno etc.

## *Column on war machines specialty*

In [218]:
df[(df['specialty'] == 'Cannon') | (df['specialty'] == 'Ballista')]

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spellbook,town,skill_creature,skill_resource
6,Jeremy,Captain,might,Cannon,Basic,Offense,Basic,Artillery,,Cove,,
19,Christian,Knight,might,Ballista,Basic,Leadership,Basic,Artillery,,Castle,,
64,Torosar,Alchemist,might,Ballista,Basic,Mysticism,Basic,Tactics,Magic Arrow,Tower,,
80,Pyre,Demoniac,might,Ballista,Basic,Logistics,Basic,Artillery,,Inferno,,
100,Ranloo,Death Knight,might,Ballista,Basic,Necromancy,Basic,Artillery,Haste,Necropolis,,
110,Arlach,Overlord,might,Ballista,Basic,Offense,Basic,Artillery,,Dungeon,,
129,Gurnisson,Barbarian,might,Ballista,Basic,Offense,Basic,Artillery,,Stronghold,,
148,Gerwulf,Beastmaster,might,Ballista,Basic,Armorer,Basic,Artillery,,Fortress,,


In [219]:
df[df['town']=='Inferno']

,name,class,class_2,specialty,level_1,skill_1,level_2,skill_2,spellbook,town,skill_creature,skill_resource
74,Calh,Demoniac,might,Gogs,Basic,Archery,Basic,Scouting,,Inferno,Gog,
75,Fiona,Demoniac,might,Hell Hounds,Advanced,Scouting,,,,Inferno,Hell Hound,
76,Ignatius,Demoniac,might,Imps,Basic,Tactics,Basic,Interference,,Inferno,Imp,
77,Marius,Demoniac,might,Demons,Advanced,Armorer,,,,Inferno,Demon,
78,Nymus,Demoniac,might,Pit Fiends,Advanced,Offense,,,,Inferno,Pit Fiend,
79,Octavia,Demoniac,might,Gold,Basic,Scholar,Basic,Offense,,Inferno,,Gold
80,Pyre,Demoniac,might,Ballista,Basic,Logistics,Basic,Artillery,,Inferno,,
81,Rashka,Demoniac,might,Efreet,Basic,Scholar,Basic,Wisdom,,Inferno,Efreeti,
82,Xeron,Demoniac,might,Devils,Basic,Leadership,Basic,Tactics,,Inferno,Devil,
83,Ash,Heretic,magic,Bloodlust,Basic,Wisdom,Basic,Eagle Eye,Bloodlust,Inferno,,
